# Introduction

SQL is based on a client-server architecture.


Python can connect with every SQL databases (Oracle MySQL, Microsoft, etc...). SQLite is a little bit particular : it is a little file, who can play the server role, without username and password. It will be helpful for this quest. But keep in mind that you could connect your Python script with other databases, following the same steps.

In [17]:
# Here we just import the SQLite file. This part is specific for SQLite.
import requests
r = requests.get('https://raw.githubusercontent.com/murpi/wilddata/master/quests/toys_and_models.sqlite')
open('toys_and_models.sqlite', 'wb').write(r.content)

307200

In [18]:
r

<Response [200]>

In [19]:
# This part is common : we have to initiate a connection between Python and the SQL Database
import sqlite3

# This is the "connector". For secured database, it will be here that  you indicate your username and password.
conn = sqlite3.connect('toys_and_models.sqlite')

# This is the "cursor". A cursor is an object that can execute a query and retrieve the content of the result.
cursor = conn.cursor()


In [20]:
# You can execute a query.
# The result is returned line by line. You can "fetch all" lines.
cursor.execute("select * from productlines").fetchall()

[('Classic Cars',
  'Attention car enthusiasts: Make your wildest car ownership dreams come true. Whether you are looking for classic muscle cars, dream sports cars or movie-inspired miniatures, you will find great choices in this category. These replicas feature superb attention to detail and craftsmanship and offer features such as working steering system, opening forward compartment, opening rear trunk with removable spare wheel, 4-wheel independent spring suspension, and so on. The models range in size from 1:10 to 1:24 scale and include numerous limited edition and several out-of-production vehicles. All models include a certificate of authenticity from their manufacturers and come fully assembled and ready for display in the home or office.',
  None,
  None),
 ('Motorcycles',
  'Our motorcycles are state of the art replicas of classic as well as contemporary motorcycle legends such as Harley Davidson, Ducati and Vespa. Models contain stunning details such as official logos, rotat

In [21]:
# Pandas can import the result of a query
import pandas as pd
pd.DataFrame(cursor.execute("select * from productlines").fetchall())

,0,1,2,3
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [22]:
# But it could be easier : you can use the pandas read_sql function.
pd.read_sql("select * from productlines", conn)

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [23]:
# Here is the schema of the database
pd.read_sql("SELECT * FROM sqlite_master where type = 'table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,customers,customers,2,CREATE TABLE `customers` (\n `customerNumber`...
1,table,employees,employees,8,CREATE TABLE `employees` (\n `employeeNumber`...
2,table,offices,offices,10,CREATE TABLE `offices` (\n `officeCode` varch...
3,table,orderdetails,orderdetails,12,CREATE TABLE `orderdetails` (\n `orderNumber`...
4,table,orders,orders,49,CREATE TABLE `orders` (\n `orderNumber` int(1...
5,table,payments,payments,57,CREATE TABLE `payments` (\n `customerNumber` ...
6,table,productlines,productlines,64,CREATE TABLE `productlines` (\n `productLine`...
7,table,products,products,66,CREATE TABLE `products` (\n `productCode` var...


![Texte alternatif…](https://mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

# It's your turn

- First you have to create a DataFrame with the 5 most ordered productcodes
- Then you have to create a second DataFrame with stocks by products
- Finally, you will make a merge (with Pandas) between both DataFrame to display the stocks of the 5 most ordered products

In [24]:
# # Your code here :
# df_orderdetails = pd.read_sql("select * from orderdetails", conn)
# df_orderdetails.drop(columns=['orderNumber','priceEach','orderLineNumber'], inplace=True)
# df_orderdetails = df_orderdetails.groupby('productCode').sum().sort_values(by='quantityOrdered', ascending=False).head(5)

In [25]:
df_orderdetails = pd.read_sql('''
                          SELECT productCode,SUM(quantityOrdered) AS quantityOrdered
                          FROM orderdetails 
                          GROUP BY productCode 
                          ORDER BY SUM(quantityOrdered) DESC LIMIT 5
                          ''', conn)

In [26]:
# df_products = pd.read_sql("select * from products", conn)
# df_products = df_products[['productCode', 'productName','quantityInStock']]

In [27]:
df_products = pd.read_sql('''
                          SELECT productCode, productName, quantityInStock 
                          FROM products  
                          '''
                          , conn)

In [28]:


df_merge = pd.merge(df_orderdetails, df_products, how='left', on='productCode')
df_merge = df_merge[['productCode', 'productName', 'quantityOrdered', 'quantityInStock']]
df_merge

,productCode,productName,quantityOrdered,quantityInStock
0,S18_3232,1992 Ferrari 360 Spider red,1561,8347
1,S18_1342,1937 Lincoln Berline,960,8693
2,S12_1108,2001 Ferrari Enzo,950,3619
3,S18_2949,1913 Ford Model T Speedster,949,4189
4,S18_4600,1940s Ford truck,947,3128


Good, but not optimized. Your "merge" with pandas is pretty the same as the "join" in SQL. Moreover, your script has to download the whole product stocks table.

So, now, you have to write a SQL query to have directly the stocks of the 5 most ordered products. (So, you have to "join" both tables in the SQL Query)

In [29]:
# Your code here :

df_merge_v2 = pd.read_sql("select p.productCode, p.productName, sum(o.quantityOrdered) as quantityOrdered, p.quantityInStock from orderdetails as o left join products as p on o.productCode = p.productCode group by o.productCode order by sum(o.quantityOrdered) desc limit 5", conn)
df_merge_v2

,productCode,productName,quantityOrdered,quantityInStock
0,S18_3232,1992 Ferrari 360 Spider red,1561,8347
1,S18_1342,1937 Lincoln Berline,960,8693
2,S12_1108,2001 Ferrari Enzo,950,3619
3,S18_2949,1913 Ford Model T Speedster,949,4189
4,S18_4600,1940s Ford truck,947,3128


In [30]:
df_merge_v2 = pd.read_sql('''
                          SELECT p.productCode, p.productName, SUM(o.quantityOrdered) AS quantityOrdered, p.quantityInStock 
                          FROM orderdetails AS o 
                          LEFT JOIN products AS p 
                                ON o.productCode = p.productCode 
                          GROUP BY o.productCode 
                          ORDER BY SUM(o.quantityOrdered) DESC LIMIT 5
                          '''
                          , conn)
df_merge_v2

,productCode,productName,quantityOrdered,quantityInStock
0,S18_3232,1992 Ferrari 360 Spider red,1561,8347
1,S18_1342,1937 Lincoln Berline,960,8693
2,S12_1108,2001 Ferrari Enzo,950,3619
3,S18_2949,1913 Ford Model T Speedster,949,4189
4,S18_4600,1940s Ford truck,947,3128


Both methods have same results ? Congrats !

# Go forward

If you want, you can try to connect to the MySQL Server with your computer. To do this, you need to have MySQL Server on your computer, and to execute your python script from your computer too (so on Jupyterlab or on Spyder, **but not on google colab**).
Below, script to connect to your MySQL Server.
And here, [the SQL script to execute](https://drive.google.com/file/d/1wXu5t7bahLhbGXOT6uCTOVLjPH-2oQ_d/view?usp=sharing) in the workbench if you don't have the database.

In [31]:
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable


In [32]:
import mysql.connector
import pandas as pd

# Please replace by your user and your password
conn = mysql.connector.connect(user='root', password='Monange1608',
                              host='localhost',
                              database='toys_and_models')

pd.read_sql('select * from orders', conn)

C:\Users\aurel\AppData\Local\Temp\ipykernel_195820\1729853189.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('select * from orders', conn)


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2021-07-23,2021-07-30,2021-07-27,Shipped,None,363
1,10101,2021-07-26,2021-08-04,2021-07-28,Shipped,Check on availability.,128
2,10102,2021-07-27,2021-08-04,2021-07-31,Shipped,None,181
3,10103,2021-08-15,2021-08-24,2021-08-19,Shipped,None,121
4,10104,2021-08-17,2021-08-26,2021-08-18,Shipped,None,141
...,...,...,...,...,...,...,...
278,10378,2023-08-28,2023-09-05,2023-08-29,Shipped,None,141
279,10379,2023-08-28,2023-09-05,2023-08-29,Shipped,None,141
280,10380,2023-09-03,2023-09-11,2023-09-05,Shipped,None,141
281,10381,2023-09-04,2023-09-12,2023-09-05,Shipped,None,321


Or you can test with this online server:

In [33]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(user='toyscie', password='WILD4Rdata!',
                              host='51.178.25.157',
                              port = '23456',
                              database='toys_and_models')

pd.read_sql('select * from orders', conn)

C:\Users\aurel\AppData\Local\Temp\ipykernel_195820\2344963350.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('select * from orders', conn)


,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2022-01-08,2022-01-15,2022-01-12,Shipped,None,363
1,10101,2022-01-11,2022-01-20,2022-01-13,Shipped,Check on availability.,128
2,10102,2022-01-12,2022-01-20,2022-01-16,Shipped,None,181
3,10103,2022-01-31,2022-02-09,2022-02-04,Shipped,None,121
4,10104,2022-02-02,2022-02-11,2022-02-03,Shipped,None,141
...,...,...,...,...,...,...,...
278,10378,2024-02-13,2024-02-21,2024-02-14,Shipped,None,141
279,10379,2024-02-13,2024-02-21,2024-02-14,Shipped,None,141
280,10380,2024-02-19,2024-02-27,2024-02-21,Shipped,None,141
281,10381,2024-02-20,2024-02-28,2024-02-21,Shipped,None,321
